In [1]:
%reset -fs

In [2]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
import pandas as pd
import datetime
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
mongo_pw = os.environ.get("MONGO_PW")

In [5]:

client = MongoClient('mongodb+srv://natedirenzo:' + mongo_pw + '@nba-db.grcwg.mongodb.net/myFirstDatabase?retryWrites=true&w=majority')


In [6]:
client.list_database_names()

['nba-data', 'admin', 'local']

In [8]:
db = client['nba-data']
cursor = db['betting-data'].find()
mongo_df = pd.DataFrame(list(cursor))
mongo_df

,_id,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,6246960a4fba57b3e39e9976,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47,0.0
1,6246960a4fba57b3e39e9977,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45,0.0
2,6246960a4fba57b3e39e9978,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45,1.0
3,6246960a4fba57b3e39e9979,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45,0.0
4,6246960a4fba57b3e39e997a,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,6246960a4fba57b3e39ed221,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46,1.0
14508,6246960a4fba57b3e39ed222,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50,0.0
14509,6246960a4fba57b3e39ed223,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46,0.0
14510,6246960a4fba57b3e39ed224,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46,1.0


In [9]:
today = datetime.datetime.today()
max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')

def get_date_list(date):
    date_list = []
    today = datetime.datetime.today()
    max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')
    delta = today - max_date
    date_list = [max_date - datetime.timedelta(days=x) for x in range(0, delta.days)]
    date_list = [d.strftime('%Y-%m-%d') for d in date_list]
    date_list.append((datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d'))
    date_list.append(datetime.datetime.today().strftime('%Y-%m-%d'))
    date_list = set(date_list)
    date_list = list(date_list)
    return date_list
    
    
    

In [10]:
date_list = get_date_list(today)


In [11]:
date_list

['2022-03-31', '2022-04-01']

In [12]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [13]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [14]:
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [15]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df

In [16]:
final_df = pd.DataFrame()

In [17]:
for i in date_list:
    get_site_rb(i)

In [18]:
driver.close()

In [19]:
final_df

,0,1,2,3,4,5,6,7,8,9
0,1 K. Kevin Love,CLE - F,@ ATL,Points,12.50 (-115)19.30Diff: 6.80 Line: 12.50...,12.5,-115,8 (under),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
1,2 B. Bobby Portis Jr.,MIL - F,@ BKN,Points,7.50 (-125)13.10Diff: 5.60 Line: 7.50 C...,7.5,-125,9 (over),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
2,3 N. Nikola Vucevic,CHI - C,vs. LAC,Points,19.50 (-120)15.00Diff: -4.50 Line: 19.5...,19.5,-120,22 (over),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
3,4 G. Goran Dragic,BKN - G,vs. MIL,Points,7.50 (-125)12.00Diff: 4.50 Line: 7.50 C...,7.5,-125,5 (under),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
4,5 P. Paul George,LAC - F,@ CHI,Points,25.50 (-130)21.20Diff: -4.30 Line: 25.5...,25.5,-130,22 (under),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
...,...,...,...,...,...,...,...,...,...,...
38,89 M. Mo Bamba,ORL - C,vs. TOR,Points,10.50 (+105)10.70Diff: 0.20 Line: 10.50...,10.5,+105 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
39,90 N. Naz Reid,MIN - C,@ DEN,Points,7.50 (EVEN)7.40Diff: -0.10 Line: 7.50 C...,7.5,EVEN TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
40,91 T. Taurean Prince,MIN - F,@ DEN,Points,9.50 (-105)9.40Diff: -0.10 Line: 9.50 C...,9.5,-105 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
41,92 D. Devin Booker,PHO - G,@ MEM,Points,27.50 (-105)27.40Diff: -0.10 Line: 27.5...,27.5,-105 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...


In [20]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [21]:
final_df.reset_index(drop=True, inplace=True)

In [22]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [23]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [24]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [25]:
bet_info = final_df.Line.str.split(" ", expand = True)
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [26]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [27]:
final_df = final_df.copy()

In [28]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [29]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [30]:
final_df['seven_day_accuracy'] = accuracies[8]

In [31]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [32]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [33]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [34]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [35]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [36]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [37]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [38]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [39]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [40]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [41]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x))

In [42]:
results = sorted_df.Result.str.split(' ', expand=True);

In [43]:
try:
    sorted_df.Result = results[3]
except:
    sorted_df.Result = results[0]

In [44]:
sorted_df = sorted_df.replace('TBD', None)

In [45]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)

In [46]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    (sorted_df['Result'].isna() == True)
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0', np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [47]:
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,2022-03-31,Bogdan,ATL,G,CLE,vs.,16.5,-115,14.8,-1.7,16.5,11.0,u,0.46,1
1,2022-04-01,Aaron Gordon,DEN,F,MIN,vs.,15.5,-115,7.0,-8.5,15.5,NaN,u,0.40,nan
2,2022-04-01,Aaron Wiggins,OKC,G,DET,vs.,15.5,-115,11.2,-4.3,15.5,NaN,u,0.40,nan
3,2022-04-01,Al Horford,BOS,C,IND,vs.,9.5,-129,12.2,2.7,9.5,NaN,o,0.40,nan
4,2022-04-01,Aleksej Pokusevski,OKC,C,DET,vs.,15.5,-149,14.2,-1.3,15.5,NaN,u,0.40,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2022-04-01,Vit Krejci,OKC,G,DET,vs.,10.5,-105,8.1,-2.4,10.5,NaN,u,0.40,nan
157,2022-03-31,Wesley Matthews,MIL,G,BKN,@,5.5,-115,5.1,-0.4,5.5,11.0,u,0.46,0
158,2022-04-01,Will Barton,DEN,G,MIN,vs.,15.5,-113,16.1,0.6,15.5,NaN,o,0.40,nan
159,2022-04-01,Zach Collins,SAS,F,POR,vs.,7.5,105,6.3,-1.2,7.5,NaN,u,0.40,nan


In [48]:
from nba_api.stats.endpoints import commonplayerinfo

In [49]:
player_info = commonplayerinfo.CommonPlayerInfo(player_id=2544)

In [50]:
player_info

In [52]:
player_info.get_data_frames()

[   PERSON_ID FIRST_NAME LAST_NAME DISPLAY_FIRST_LAST DISPLAY_LAST_COMMA_FIRST  \
 0       2544     LeBron     James       LeBron James            James, LeBron   
 
   DISPLAY_FI_LAST   PLAYER_SLUG            BIRTHDATE  \
 0        L. James  lebron-james  1984-12-30T00:00:00   
 
                          SCHOOL COUNTRY  ...    PLAYERCODE FROM_YEAR TO_YEAR  \
 0  St. Vincent-St. Mary HS (OH)     USA  ...  lebron_james      2003    2021   
 
    DLEAGUE_FLAG NBA_FLAG GAMES_PLAYED_FLAG DRAFT_YEAR DRAFT_ROUND  \
 0             N        Y                 Y       2003           1   
 
    DRAFT_NUMBER GREATEST_75_FLAG  
 0             1                Y  
 
 [1 rows x 33 columns],
    PLAYER_ID   PLAYER_NAME TimeFrame   PTS  AST  REB   PIE
 0       2544  LeBron James   2021-22  30.1  6.3  8.2  0.18,
    SEASON_ID
 0      12003
 1      22003
 2      12004
 3      22004
 4      32004
 ..       ...
 67     42020
 68     52020
 69     12021
 70     22021
 71     32021
 
 [72 rows x 1 columns]]